# Imports

In [97]:
import pandas as pd 
import numpy as np 
import tensorflow 
import keras 
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer 
from tqdm.auto import tqdm
import re
import nltk 
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.corpus import wordnet
import spacy
from tensorflow.keras.preprocessing.text import Tokenizer

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Honda\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Read_Data

In [98]:
Data=pd.read_csv('dataset.csv')
Data_prep_2=Data.copy()

In [99]:
Data.drop(columns='id',inplace=True)

In [100]:
Data.head()

,label,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation


# EDA

Check_Counts

In [101]:
Data.label.value_counts()

0    29720
1     2242
Name: label, dtype: int64

Check_nulls

In [102]:
Data.isna().any()

label    False
tweet    False
dtype: bool

Check_duplicates

In [103]:
Data.duplicated().any()

True

In [104]:
Data.drop_duplicates(inplace=True)

# Preprocessing

As baseline processing we will do so :

1-check for mails to remove

2-check for websites to remove
 
3-remove non chars

4-Normalization

5-remove stop words

6-Lemmitization

In [105]:
Data['tweet'].apply(lambda x: re.findall('\S+@\S+',x)).any()

True

In [106]:
Data['tweet']=Data['tweet'].apply(lambda x: re.sub('\S+@\S+',' ',x))
Data['tweet']

0         @user when a father is dysfunctional and is s...
1        @user @user thanks for #lyft credit i can't us...
2                                      bihday your majesty
3        #model   i love u take with u all the time in ...
4                   factsguide: society now    #motivation
                               ...                        
31956    off fishing tomorrow @user carnt wait first ti...
31957    ate @user isz that youuu?ðððððð...
31958      to see nina turner on the airwaves trying to...
31959    listening to sad songs on a monday morning otw...
31961                     thank you @user for you follow  
Name: tweet, Length: 29530, dtype: object

In [107]:
Data['tweet'].apply(lambda x: re.findall('http\S+',x)).any()

False

In [108]:
Data['tweet']=Data['tweet'].apply(lambda x:re.sub('[^A-Za-z0-9]',' ',x))
Data['tweet']

0          user when a father is dysfunctional and is s...
1         user  user thanks for  lyft credit i can t us...
2                                      bihday your majesty
3         model   i love u take with u all the time in ...
4                   factsguide  society now     motivation
                               ...                        
31956    off fishing tomorrow  user carnt wait first ti...
31957    ate  user isz that youuu                      ...
31958      to see nina turner on the airwaves trying to...
31959    listening to sad songs on a monday morning otw...
31961                     thank you  user for you follow  
Name: tweet, Length: 29530, dtype: object

In [109]:
Data['tweet']=Data['tweet'].apply(lambda x: x.lower())

In [110]:
Data['tweet']

0          user when a father is dysfunctional and is s...
1         user  user thanks for  lyft credit i can t us...
2                                      bihday your majesty
3         model   i love u take with u all the time in ...
4                   factsguide  society now     motivation
                               ...                        
31956    off fishing tomorrow  user carnt wait first ti...
31957    ate  user isz that youuu                      ...
31958      to see nina turner on the airwaves trying to...
31959    listening to sad songs on a monday morning otw...
31961                     thank you  user for you follow  
Name: tweet, Length: 29530, dtype: object

In [111]:
Data['tweet']=Data['tweet'].apply(lambda x: re.sub('\d+','',x))
Data['tweet']

0          user when a father is dysfunctional and is s...
1         user  user thanks for  lyft credit i can t us...
2                                      bihday your majesty
3         model   i love u take with u all the time in ...
4                   factsguide  society now     motivation
                               ...                        
31956    off fishing tomorrow  user carnt wait first ti...
31957    ate  user isz that youuu                      ...
31958      to see nina turner on the airwaves trying to...
31959    listening to sad songs on a monday morning otw...
31961                     thank you  user for you follow  
Name: tweet, Length: 29530, dtype: object

In [112]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words=stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Honda\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [113]:
Data['tweet']=Data['tweet'].apply(lambda x : [word for word in x.split()  if word not in stop_words])
Data['tweet']

0        [user, father, dysfunctional, selfish, drags, ...
1        [user, user, thanks, lyft, credit, use, cause,...
2                                        [bihday, majesty]
3                      [model, love, u, take, u, time, ur]
4                        [factsguide, society, motivation]
                               ...                        
31956    [fishing, tomorrow, user, carnt, wait, first, ...
31957                              [ate, user, isz, youuu]
31958    [see, nina, turner, airwaves, trying, wrap, ma...
31959    [listening, sad, songs, monday, morning, otw, ...
31961                                [thank, user, follow]
Name: tweet, Length: 29530, dtype: object

In [114]:
pos_tags=[wordnet.VERB,wordnet.ADJ,wordnet.ADV,wordnet.NOUN]
lemmitaizer=WordNetLemmatizer()
for pos in pos_tags:
    Data['tweet']=Data['tweet'].apply(lambda x: [lemmitaizer.lemmatize(word,pos=pos) for word in x])
    
Data['tweet']

0        [user, father, dysfunctional, selfish, drag, k...
1        [user, user, thank, lyft, credit, use, cause, ...
2                                        [bihday, majesty]
3                      [model, love, u, take, u, time, ur]
4                        [factsguide, society, motivation]
                               ...                        
31956    [fish, tomorrow, user, carnt, wait, first, tim...
31957                              [eat, user, isz, youuu]
31958    [see, nina, turner, airwave, try, wrap, mantle...
31959    [listen, sad, song, monday, morning, otw, work...
31961                                [thank, user, follow]
Name: tweet, Length: 29530, dtype: object

In [115]:
Data['tweet']=Data['tweet'].apply(lambda x: ' '.join(x))

In [116]:
Data['tweet']

0        user father dysfunctional selfish drag kid dys...
1        user user thank lyft credit use cause offer wh...
2                                           bihday majesty
3                              model love u take u time ur
4                            factsguide society motivation
                               ...                        
31956        fish tomorrow user carnt wait first time year
31957                                   eat user isz youuu
31958    see nina turner airwave try wrap mantle genuin...
31959          listen sad song monday morning otw work sad
31961                                    thank user follow
Name: tweet, Length: 29530, dtype: object

In [117]:
Data.to_csv("tweets_preprocessed.csv")

# Feature extraction

1-count vectorizer

2-tfidf

3-tokenizer(binary,count,freq)

4-pretrained(glove)

In [118]:
from sklearn.model_selection import train_test_split
target=Data['label']
x_train, x_test ,y_train ,y_test = train_test_split(Data['tweet'],target,test_size=0.1,stratify=target)

In [119]:
x_train=x_train.tolist()

In [23]:
count_vect=CountVectorizer(max_features=10000,ngram_range=(1,1))
count_vect.fit(x_train)

CountVectorizer(max_features=10000)

In [24]:
count_vect.vocabulary_['york']

9959

In [25]:
len(count_vect.get_feature_names())

C:\Users\Honda\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


10000

In [26]:
X_train_count=count_vect.transform(x_train).todense()
X_test_count=count_vect.transform(x_test).todense()

In [27]:
pd.DataFrame(X_train_count,index=x_train,columns=count_vect.get_feature_names())

C:\Users\Honda\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aap,aaron,ab,abandon,abasel,abba,abc,abe,ability,abitur,...,zionazis,zionism,zombie,zomg,zone,zoo,zootopia,zoro,zuma,zzzzzzzz
liv know weird yeahhh bread get hole stoner sistabanta,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
one today believe bihday timeflies,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
travel sunday family travel couple family life chiangmai thailand,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
leisure time day leisure leisuretime bangsean milksshop cocoa bos,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
love findingdory wait movie even find dory,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
thankful taxi thankful positive,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
life measure number breath take moment take breath away maya angelou love woman,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
everyday caturday caturday cat saturday indraloka weekend,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
reason millennials work keith breene user challenge employment,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
print(X_train_count.shape)
print(X_test_count.shape)

(26577, 10000)
(2953, 10000)


In [29]:
from keras import layers,models
model_1=models.Sequential()
model_1.add(layers.Dense(128,activation='relu',input_shape=(10000,)))
model_1.add(layers.Dense(64,activation='relu'))
model_1.add(layers.Dense(32,activation='relu'))
model_1.add(layers.Dense(1,activation='sigmoid'))

In [30]:
from tensorflow.keras.optimizers import RMSprop
model_1.compile(optimizer= RMSprop(lr=0.0001),
              loss= keras.losses.binary_crossentropy,
              metrics= [keras.metrics.binary_accuracy,keras.metrics.AUC()])

C:\Users\Honda\anaconda3\lib\site-packages\keras\optimizer_v2\rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [32]:
model_1.fit(X_train_count,y_train,epochs=10,validation_data=(X_test_count,y_test))

Epoch 1/10
831/831 [==============================] - 14s 17ms/step - loss: 0.1083 - binary_accuracy: 0.9666 - auc: 0.9442 - val_loss: 0.1557 - val_binary_accuracy: 0.9553 - val_auc: 0.8791
Epoch 2/10
831/831 [==============================] - 13s 16ms/step - loss: 0.1035 - binary_accuracy: 0.9689 - auc: 0.9477 - val_loss: 0.1524 - val_binary_accuracy: 0.9570 - val_auc: 0.8901
Epoch 3/10
831/831 [==============================] - 15s 18ms/step - loss: 0.0984 - binary_accuracy: 0.9708 - auc: 0.9517 - val_loss: 0.1522 - val_binary_accuracy: 0.9577 - val_auc: 0.8869
Epoch 4/10
831/831 [==============================] - 13s 16ms/step - loss: 0.0935 - binary_accuracy: 0.9725 - auc: 0.9557 - val_loss: 0.1492 - val_binary_accuracy: 0.9594 - val_auc: 0.8946
Epoch 5/10
831/831 [==============================] - 13s 16ms/step - loss: 0.0900 - binary_accuracy: 0.9742 - auc: 0.9577 - val_loss: 0.1509 - val_binary_accuracy: 0.9583 - val_auc: 0.8977
Epoch 6/10
831/831 [==============================

# Tokenizer

In [33]:
tokenizer=Tokenizer(num_words=10000,oov_token='<OOV>')
tokenizer.fit_on_texts(x_train)

In [34]:
tokenizer.word_index

{'<OOV>': 1,
 'user': 2,
 'day': 3,
 'love': 4,
 'get': 5,
 'happy': 6,
 'amp': 7,
 'go': 8,
 'make': 9,
 'life': 10,
 'today': 11,
 'like': 12,
 'u': 13,
 'new': 14,
 'good': 15,
 'father': 16,
 'see': 17,
 'time': 18,
 'smile': 19,
 'people': 20,
 'one': 21,
 'bihday': 22,
 'friend': 23,
 'come': 24,
 'feel': 25,
 'look': 26,
 'work': 27,
 'want': 28,
 'wait': 29,
 'girl': 30,
 'weekend': 31,
 'thank': 32,
 'week': 33,
 'fun': 34,
 'think': 35,
 'family': 36,
 'summer': 37,
 'need': 38,
 'know': 39,
 'year': 40,
 'great': 41,
 'say': 42,
 'thankful': 43,
 'live': 44,
 'friday': 45,
 'positive': 46,
 'first': 47,
 'morning': 48,
 'world': 49,
 'beautiful': 50,
 'back': 51,
 'tomorrow': 52,
 'best': 53,
 'thing': 54,
 'dad': 55,
 'take': 56,
 'watch': 57,
 'really': 58,
 'way': 59,
 'even': 60,
 'home': 61,
 'sad': 62,
 'orlando': 63,
 'never': 64,
 'fathersday': 65,
 'blog': 66,
 'night': 67,
 'sunday': 68,
 'music': 69,
 'trump': 70,
 'happiness': 71,
 'right': 72,
 'find': 73,
 'cut

In [35]:
x_train_tokenized=tokenizer.texts_to_matrix(x_train,mode='binary')
x_test_tokenized=tokenizer.texts_to_matrix(x_test,mode='binary')

In [36]:
pd.DataFrame(x_train_tokenized,index=x_train)

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
liv know weird yeahhh bread get hole stoner sistabanta,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
one today believe bihday timeflies,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
travel sunday family travel couple family life chiangmai thailand,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
leisure time day leisure leisuretime bangsean milksshop cocoa bos,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
love findingdory wait movie even find dory,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
thankful taxi thankful positive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
life measure number breath take moment take breath away maya angelou love woman,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
everyday caturday caturday cat saturday indraloka weekend,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
reason millennials work keith breene user challenge employment,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
from keras import layers,models
model_2=models.Sequential()
model_2.add(layers.Dense(128,activation='relu',input_shape=(10000,)))
model_2.add(layers.Dense(64,activation='relu'))
model_2.add(layers.Dense(32,activation='relu'))
model_2.add(layers.Dense(1,activation='sigmoid'))

In [39]:
from tensorflow.keras.optimizers import RMSprop,Adam
model_2.compile(optimizer= RMSprop(lr=0.0005),
              loss= keras.losses.binary_crossentropy,
              metrics= [keras.metrics.binary_accuracy,keras.metrics.AUC()])

C:\Users\Honda\anaconda3\lib\site-packages\keras\optimizer_v2\rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [40]:
model_2.fit(x_train_tokenized,y_train,epochs=10,validation_data=(x_test_tokenized,y_test))

Epoch 1/10
831/831 [==============================] - 15s 17ms/step - loss: 0.2101 - binary_accuracy: 0.9345 - auc_1: 0.8114 - val_loss: 0.1822 - val_binary_accuracy: 0.9441 - val_auc_1: 0.8525
Epoch 2/10
831/831 [==============================] - 14s 17ms/step - loss: 0.1478 - binary_accuracy: 0.9536 - auc_1: 0.9041 - val_loss: 0.1663 - val_binary_accuracy: 0.9523 - val_auc_1: 0.8678
Epoch 3/10
831/831 [==============================] - 13s 16ms/step - loss: 0.1208 - binary_accuracy: 0.9637 - auc_1: 0.9293 - val_loss: 0.1537 - val_binary_accuracy: 0.9550 - val_auc_1: 0.8903
Epoch 4/10
831/831 [==============================] - 14s 16ms/step - loss: 0.1007 - binary_accuracy: 0.9705 - auc_1: 0.9452 - val_loss: 0.1517 - val_binary_accuracy: 0.9567 - val_auc_1: 0.9051
Epoch 5/10
831/831 [==============================] - 13s 16ms/step - loss: 0.0852 - binary_accuracy: 0.9764 - auc_1: 0.9570 - val_loss: 0.1683 - val_binary_accuracy: 0.9539 - val_auc_1: 0.9093
Epoch 6/10
831/831 [==========

# Glove

In [120]:
nlp = spacy.load('en_core_web_md')

In [121]:
x_train_v=np.zeros((len(x_train),300))
x_test_v=np.zeros((len(x_test),300))

In [122]:
for i, doc in enumerate(nlp.pipe(x_train)):
    x_train_v[i, :] = doc.vector

for i, doc in enumerate(nlp.pipe(x_test)):
    x_test_v[i, :] = doc.vector

In [44]:
from keras import layers,models
model_3=models.Sequential()
model_3.add(layers.Dense(128,activation='relu',input_shape=(300,)))
model_3.add(layers.Dense(64,activation='relu'))
model_3.add(layers.Dense(32,activation='relu'))
model_3.add(layers.Dense(1,activation='sigmoid'))

In [46]:
from tensorflow.keras.optimizers import RMSprop,Adam
model_3.compile(optimizer= RMSprop(lr=0.0005),
              loss= keras.losses.binary_crossentropy,
              metrics= [keras.metrics.binary_accuracy,keras.metrics.AUC()])

In [47]:
model_3.fit(x_train_v,y_train,epochs=10,validation_data=(x_test_v,y_test))

Epoch 1/10
831/831 [==============================] - 3s 3ms/step - loss: 0.1914 - binary_accuracy: 0.9368 - auc_2: 0.8398 - val_loss: 0.1551 - val_binary_accuracy: 0.9451 - val_auc_2: 0.9060
Epoch 2/10
831/831 [==============================] - 2s 2ms/step - loss: 0.1624 - binary_accuracy: 0.9444 - auc_2: 0.8903 - val_loss: 0.1479 - val_binary_accuracy: 0.9475 - val_auc_2: 0.9167
Epoch 3/10
831/831 [==============================] - 2s 2ms/step - loss: 0.1535 - binary_accuracy: 0.9474 - auc_2: 0.9053 - val_loss: 0.1549 - val_binary_accuracy: 0.9468 - val_auc_2: 0.9132
Epoch 4/10
831/831 [==============================] - 2s 2ms/step - loss: 0.1474 - binary_accuracy: 0.9492 - auc_2: 0.9131 - val_loss: 0.1409 - val_binary_accuracy: 0.9516 - val_auc_2: 0.9230
Epoch 5/10
831/831 [==============================] - 2s 2ms/step - loss: 0.1414 - binary_accuracy: 0.9510 - auc_2: 0.9210 - val_loss: 0.1433 - val_binary_accuracy: 0.9506 - val_auc_2: 0.9268
Epoch 6/10
831/831 [====================

# CNN_N_grams

In [54]:
def get_longest_sentence(data):
    max_len=0
    for text in data:
        text_len=len(text.split())
        max_len=max(text_len,max_len)
        
    return max_len

In [55]:
longest_input=get_longest_sentence(Data['tweet'])

In [56]:
longest_input

37

In [57]:
data_emb = np.zeros((len(Data['tweet']), longest_input, 300))
for i, text in enumerate(tqdm(nlp.pipe(Data['tweet']), total=len(Data['tweet']))):
    for j, token in enumerate(text):
        data_emb[i, j] = token.vector

#shape of data embedded (no of instances , max length of each sentence, 300 vct)

In [58]:
import tensorflow as tf

# define the network
inputs = tf.keras.layers.Input((longest_input, 300))
reshaped = tf.keras.layers.Reshape((longest_input, 300, 1))(inputs)


filters = [2, 3, 4]

# define the conv net
conv_1 = tf.keras.layers.Conv2D(100, (filters[0], 300), activation='relu')(reshaped)
conv_2 = tf.keras.layers.Conv2D(100, (filters[1], 300), activation='relu')(reshaped)
conv_3 = tf.keras.layers.Conv2D(100, (filters[2], 300), activation='relu')(reshaped)

# define max-pooling
pool_1 = tf.keras.layers.MaxPooling2D((longest_input - filters[0] + 1, 1), strides=(1,1))(conv_1)
pool_2 = tf.keras.layers.MaxPooling2D((longest_input - filters[1] + 1, 1), strides=(1,1))(conv_2)
pool_3 = tf.keras.layers.MaxPooling2D((longest_input - filters[2] + 1, 1), strides=(1,1))(conv_3)

# concatenate the convs
merged_tensor = tf.keras.layers.concatenate([pool_1, pool_2, pool_3], axis=1)

# now flatten them and add a dense layer
flatten = tf.keras.layers.Flatten()(merged_tensor)

# add a dense layer
clf = tf.keras.layers.Dense(100, activation='relu')(flatten)

# add final output
clf = tf.keras.layers.Dense(1, activation='sigmoid')(clf)

In [61]:
model = tf.keras.models.Model(inputs, clf)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc','AUC'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 37, 300)]    0           []                               
                                                                                                  
 reshape (Reshape)              (None, 37, 300, 1)   0           ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 36, 1, 100)   60100       ['reshape[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 35, 1, 100)   90100       ['reshape[0][0]']                
                                                                                            

In [62]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data_emb, Data['label'], test_size=.2)

In [63]:
model.fit(x_train, y_train, epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
739/739 [==============================] - 23s 29ms/step - loss: 0.0000e+00 - acc: 0.9309 - auc: 0.4980 - val_loss: 0.0000e+00 - val_acc: 0.9306 - val_auc: 0.5000
Epoch 2/10
739/739 [==============================] - 23s 31ms/step - loss: 0.0000e+00 - acc: 0.9321 - auc: 0.5000 - val_loss: 0.0000e+00 - val_acc: 0.9306 - val_auc: 0.5000
Epoch 3/10
739/739 [==============================] - 23s 31ms/step - loss: 0.0000e+00 - acc: 0.9321 - auc: 0.5000 - val_loss: 0.0000e+00 - val_acc: 0.9306 - val_auc: 0.5000
Epoch 4/10
739/739 [==============================] - 25s 34ms/step - loss: 0.0000e+00 - acc: 0.9321 - auc: 0.5000 - val_loss: 0.0000e+00 - val_acc: 0.9306 - val_auc: 0.5000
Epoch 5/10
739/739 [==============================] - 26s 35ms/step - loss: 0.0000e+00 - acc: 0.9321 - auc: 0.5000 - val_loss: 0.0000e+00 - val_acc: 0.9306 - val_auc: 0.5000
Epoch 6/10
739/739 [==============================] - 24s 32ms/step - loss: 0.0000e+00 - acc: 0.9321 - auc: 0.5000 - val_loss: 0.0